# 1 CARGAR EL MODELO PRE ENTRENADO

In [ ]:
from sagemaker import get_execution_role
from sagemaker import Session
role = get_execution_role()
#role = 'arn:aws:iam::095498312899:role/service-role/AmazonSageMaker-ExecutionRole-20231026T100283';
sess = Session()
bucket = sess.default_bucket()

import tensorflow as tf
import keras
from keras.models import load_model
tf_framework_version = tf.__version__

import h5py
import numpy as np
import os
import tarfile

from sagemaker.tensorflow.serving import Model

In [ ]:
imput_bucket = "pteriscope"
prefix = "PterygiumDetectionModel"
filename = "model.zip"

In [19]:
import numpy as np
import tensorflow as tf
from keras.models import model_from_json

# if your model is saved as only a .hdf5 file
MODEL_LOCATION =''

# or if your model is saved as 2 files: model as a .json file, and weights as a .h5 file
JSON_LOCATION = 'saved_models/ResNext50/resnext50_model.json'
WEIGHTS_LOCATION = 'saved_models/ResNext50/resnext50_weights_.hdf5'

In [5]:
with open(JSON_LOCATION, 'r') as json_file:
    json_savedModel= json_file.read()
    json_file.close()

# Cargamos la arquitectura del modelo
model = tf.keras.models.model_from_json(json_savedModel)
model.load_weights(WEIGHTS_LOCATION)
model.compile(optimizer = 'SGD', loss = "categorical_crossentropy", metrics = ["accuracy"])
print("loaded model from JSON_LOCATION and WEIGHTS_LOCATION")

loaded model from JSON_LOCATION and WEIGHTS_LOCATION


# 2 CONVERTIR EL MODELO AL FORMATO QUE PUEDA LEER AWS

- Converts to a Protobuff file
- Saves it in a certain aws file structure
- Tarballs this file and zips it

In [10]:
def convert_h5_to_aws(loaded_model):
    """
    given a pre-trained keras model, this function converts it to a TF protobuf format
    and saves it in the file structure which aws expects
    """

    from tensorflow.python.saved_model import builder
    from tensorflow.python.saved_model.signature_def_utils import predict_signature_def
    from tensorflow.python.saved_model import tag_constants

    # This is the file structure which AWS expects. Cannot be changed.
    model_version = '1'
    export_dir = 'export/Servo/' + model_version

    # Build the Protocol Buffer SavedModel at 'export_dir'
    builder = builder.SavedModelBuilder(export_dir)

    # Create prediction signature to be used by TensorFlow Serving Predict API
    signature = predict_signature_def(
        inputs={"inputs": loaded_model.input}, outputs={"score": loaded_model.output})

    from keras import backend as K
    with K.get_session() as sess:
        # Save the meta graph and variables
        builder.add_meta_graph_and_variables(
            sess=sess, tags=[tag_constants.SERVING], signature_def_map={"serving_default": signature})
        builder.save()

    #create a tarball/tar file and zip it
    import tarfile
    with tarfile.open('model.tar.gz', mode='w:gz') as archive:
        archive.add('export', recursive=True)

tf.compat.v1.disable_eager_execution()
convert_h5_to_aws(model)

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: export/Servo/1\saved_model.pb


# MOVER EL ARCHIVO TARBALL (TAR.GZ) A S3

In [11]:
import sagemaker
import boto3, re

sagemaker_session = sagemaker.Session(boto_session=boto3.Session(region_name='us-east-1'))
#inputs = sagemaker_session.upload_data(path='model.tar.gz', key_prefix='model')

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\Acer\AppData\Local\sagemaker\sagemaker\config.yaml


In [12]:
# where did it upload to?
print("Bucket name is:")
sagemaker_session.default_bucket()

Bucket name is:


'sagemaker-us-east-1-095498312899'

In [14]:
from sagemaker import get_execution_role

# the (default) IAM role you created when creating this notebook
role = 'arn:aws:iam::095498312899:role/service-role/AmazonSageMaker-ExecutionRole-20231026T100283'

# Create a Sagemaker model (see AWS console>SageMaker>Models)
from sagemaker.tensorflow.model import TensorFlowModel
sagemaker_model = TensorFlowModel(model_data = 's3://' + sagemaker_session.default_bucket() + '/model/model.tar.gz',
                                  role = role,
                                  framework_version = '2.10.0',
                                  entry_point = 'train.py')

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\Acer\AppData\Local\sagemaker\sagemaker\config.yaml


In [15]:
# Deploy a SageMaker to an endpoint
predictor = sagemaker_model.deploy(initial_instance_count=1,
                                   instance_type='ml.t2.medium')

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\Acer\AppData\Local\sagemaker\sagemaker\config.yaml
-----!

In [16]:
# What is our endpoint called?
endpoint = predictor.endpoint_name
endpoint

'tensorflow-inference-2023-10-28-23-58-41-631'

In [17]:
# Create a predictor which uses this new endpoint
import sagemaker
from sagemaker.tensorflow.model import TensorFlowModel

#endpoint = 'tensorflow-inference-2023-10-28-17-25-56-408' #get endpoint name from SageMaker > endpoints

endpoint_predictor = sagemaker.tensorflow.model.TensorFlowPredictor(endpoint, sagemaker_session)

In [21]:
import cv2

image_test = cv2.imread('tests/GRAVE.jpg')
resize = tf.image.resize(image_test, (224, 224))
expanded = np.expand_dims(resize, 0)

In [24]:
yhat = predictor.predict(expanded)
yhat

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received client error (400) from primary with message "{    "error": "Session was not created with a graph before Run()!"}". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/tensorflow-inference-2023-10-28-23-58-41-631 in account 095498312899 for more information.